<a href="https://colab.research.google.com/github/zariyagh/SimPep_and_OP-AND/blob/main/SimPep_Complet_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuration file creation

In [24]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Input
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import plot_model
import sys
import torch
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel
import re
import numpy as np
import os
import requests
from tqdm.auto import tqdm


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Functions

In [3]:
#
def makedatasetPos(data):
  dataset=[]
  label=[]
  for i in range(len(data)):
    for j in range(i+1,len(data)):
      dataset.append(np.concatenate((data[i],data[j]), axis=None))
      label.append(1)
      #dataset.append(np.concatenate((data[j],data[i]), axis=None))
      #label.append(1)
  return(dataset,label)

In [4]:

def makedatasetNeg(data1,data2):
  dataset=[]
  label=[]
  for i in range(len(data1)):
    for j in range(len(data2)):
      dataset.append(np.concatenate((data1[i],data2[j]), axis=None))
      label.append(0)
      #dataset.append(np.concatenate((data2[j],data1[i]), axis=None))
      #label.append(0)
  return(dataset,label)

In [5]:
def InternalTest(x1_test,x0_test):
  dataset_test2=[]    #concat test
  label_test2=[]
  for i in range(len(x0_test)):
    for j in range(i+1,len(x0_test)):
      dataset_test2.append(np.concatenate((x0_test[i],x0_test[j]), axis=None))
      label_test2.append(1)

  for i in range(len(x1_test)):
    for j in range(i+1,len(x1_test)):
      dataset_test2.append(np.concatenate((x1_test[i],x1_test[j]), axis=None))
      label_test2.append(1)

  for i in range(len(x0_test)):
    for j in range(len(x1_test)):
      dataset_test2.append(np.concatenate((x0_test[i],x1_test[j]), axis=None))
      label_test2.append(0)
  return(np.asarray(dataset_test2),np.asarray(label_test2))


In [6]:
def MakeDatasetforTrain(P1,P0,N,LP1,LP0,LN):
  TempP0=[]
  TempL0=[]
  i=0
  while(i<len(P1)):
    l=random.randint(0,len(P0)-1)
    TempP0.append(P0[l])
    TempL0.append(LP0[l])
    i=i+1
  print('P1',len(P1), 'P0', len(TempP0))
  train=np.copy(np.append(P1,TempP0,axis=0))
  label=np.copy(np.append(LP1,TempL0))

  TempN=[]
  TempLN=[]
  while(len(TempN)<len(train)):
    l=random.randint(0,len(N)-1)
    TempN.append(N[l])
    TempLN.append(LN[l])

  train=np.copy(np.append(train,TempN,axis=0))
  label=np.copy(np.append(label,TempLN))

  return(train,label)


In [7]:
def ExternalTest(x0_test,x1_test,x0_train,x1_train):
  dataset_test1=[] # concat train
  label_test1=[]
  Indexsample=[]
  for i in range(len(x0_test)):
    for j in range(len(x0_train)):
      dataset_test1.append(np.concatenate((x0_test[i],x0_train[j]), axis=None))
      label_test1.append(1)
      Indexsample.append(str(i)+",0,0")   #index in test,x0_test,x0_train
  for i in range(len(x0_test)):
    for j in range(len(x1_train)):
      dataset_test1.append(np.concatenate((x0_test[i],x1_train[j]), axis=None))
      label_test1.append(0)
      Indexsample.append(str(i)+",0,1")   #index in test,x0_test,x1_train
  for i in range(len(x1_test)):
    for j in range(len(x0_train)):
      dataset_test1.append(np.concatenate((x1_test[i],x0_train[j]), axis=None))
      label_test1.append(0)
      Indexsample.append(str(i)+",1,0")  #index in test,x1_test,x0_train
  for i in range(len(x1_test)):
    for j in range(len(x1_train)):
      dataset_test1.append(np.concatenate((x1_test[i],x1_train[j]), axis=None))
      label_test1.append(1)
      Indexsample.append(str(i)+",1,1")  #index in test,x1_test,x1_train
  return(np.asarray(dataset_test1),np.asarray(label_test1),Indexsample)

In [8]:
def DefineTest2(x0_test,x1_test,Indexsample,y_pred):
  Pred=[]
  Label=[]
  for i in range(len(x0_test)):
    cnt0Pred=0
    cnt1Pred=0
    cnt1=0
    cnt0=0
    for j in range(len(Indexsample)):
      triplet=Indexsample[j].split(",")   #pair[0] is index from test, pair[1] identifies which test x0_test or s1_test, pair[2] identifies which x0
      if int(triplet[0])==i: #intdex test
        if int(triplet[1])==0: #type test
          if int(triplet[2])==1: #type train
              cnt1Pred=cnt1Pred+y_pred[j]
              cnt1=cnt1+1
          if int(triplet[2])==0: #type train
              cnt0Pred=cnt0Pred+y_pred[j]
              cnt0=cnt0+1
    pos=(1-(cnt0Pred/cnt0))+(cnt1Pred/cnt1)
    neg=(cnt0Pred/cnt0)+(1-(cnt1Pred/cnt1))
    Pred.append(pos/(pos+neg))
    Label.append(0)

  for i in range(len(x1_test)):
    cnt0Pred=0
    cnt1Pred=0
    cnt1=0
    cnt0=0
    for j in range(len(Indexsample)):
      triplet=Indexsample[j].split(",")   #pair[0] is index from test, pair[1] identifies which test x0_test or s1_test, pair[2] identifies which x0
      if int(triplet[0])==i: #intdex test
        if int(triplet[1])==1: #type test
          if int(triplet[2])==0: #type train
              cnt0Pred=cnt0Pred+y_pred[j]
              cnt0=cnt0+1
          if int(triplet[2])==1: #type train
              cnt1Pred=cnt1Pred+y_pred[j]
              cnt1=cnt1+1
    pos=(1-(cnt0Pred/cnt0))+(cnt1Pred/cnt1)
    neg=(cnt0Pred/cnt0)+(1-(cnt1Pred/cnt1))
    Pred.append(pos/(pos+neg))
    Label.append(1)
  return(Pred,Label)


## OPD Prediction


In [9]:
def RealTest(X,x0_train,x1_train):
  dataset_test=[] # concat train
  Indexsample=[]
  for i in range(len(X)):
    for j in range(len(x0_train)):
      dataset_test.append(np.concatenate((X[i],x0_train[j]), axis=None))
      Indexsample.append(str(i)+",0")   #index in test,x0_test,x0_train
  for i in range(len(X)):
    for j in range(len(x1_train)):
      dataset_test.append(np.concatenate((X[i],x1_train[j]), axis=None))
      Indexsample.append(str(i)+",1")   #index in test,x0_test,x1_train
  return(np.asarray(dataset_test),Indexsample)


In [10]:
def RealPredict(X,test_pred,Indexsample):

  Pred=[]
  for i in range(len(X)):
    cnt0Pred=0
    cnt1Pred=0
    cnt1=0
    cnt0=0
    for j in range(len(Indexsample)):
      triplet=Indexsample[j].split(",")   #pair[0] is index from test, pair[1] identifies which test x0_test or s1_test, pair[2] identifies which x0

      if int(triplet[0])==i: #intdex test
          if int(triplet[1])==1: #type train
            cnt1Pred=cnt1Pred+test_pred[j]
            cnt1=cnt1+1
          if int(triplet[1])==0: #type train
            cnt0Pred=cnt0Pred+test_pred[j]
            cnt0=cnt0+1
    pos=(1-(cnt0Pred/cnt0))+(cnt1Pred/cnt1)
    neg=(cnt0Pred/cnt0)+(1-(cnt1Pred/cnt1))
    Pred.append(pos/(pos+neg))
  return(Pred)

In [11]:

def Accuracy(label_test, pred):

  print('AUC: ', metrics.roc_auc_score(label_test, pred), 'AUPR: ', metrics.average_precision_score(label_test, pred))
  for i  in range(len(pred)):
    if pred[i]>=0.5:
      pred[i]=1
    else:
      pred[i]=0
  tn, fp, fn, tp = confusion_matrix(label_test, pred).ravel()
  specificity = tn / (tn+fp)
  sensitivity = tp / (tp + fn)
  print('ACC:',metrics.accuracy_score(label_test, pred)*100,'Spc:',specificity,'Sen: ', sensitivity,'TN:',tn, 'FP: ', fp, 'FN',fn, 'TP',tp)


In [12]:
def SelectName(Pred,cutoff):
  # dir3='/content/drive/MyDrive/ThesisStudent/Osta/21-4-1402/Abbass/After_Abbass_death/'
  # f=open(dir3+'pep_name.txt')
  f = open('/content/drive/MyDrive/Osta/camel_df.csv')
  i=0
  dic1={}
  lines = f.readlines()
  for i in range(0,len(lines)):
        line=lines[i].strip().split(",")[0]
        if Pred[i]>=cutoff:
          dic1[line]=Pred[i]
          print(line , Pred[i])
        i=i+1
  print(dic1)
  f.close()

# ProtBERT Encoding

In [ ]:
! pip install Bio
! pip install tensorflow
! pip install seqIO
! pip install biopython
! pip install transformers
! pip install -q biopython
! pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.1/321.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 3.4 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
    

### Model loading

In [ ]:
!test -d bertviz_repo && echo "FYI: bertviz_repo directory already exists, to pull latest version uncomment this line: !rm -r bertviz_repo"
# !rm -r bertviz_repo # Uncomment if you need a clean pull from repo
!test -d bertviz_repo || git clone https://github.com/jessevig/bertviz bertviz_repo
if not 'bertviz_repo' in sys.path:
  sys.path += ['bertviz_repo']
!pip install regex

In [ ]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
model = BertModel.from_pretrained("Rostlab/prot_bert")

### Data preparation

In [ ]:
def data_prepared(file_name):
  f = open(file_name, 'r')
  proteins_name=[]
  my_seq =[]
  with f:
    for row in f:
      row = row.strip()
      proteins_name.append(row)
      new_r = ' '.join(row)
      my_seq.append(new_r)
  return my_seq, proteins_name

In [ ]:
file_name = 'YOUR FILE of PROTEIN SEQUENCES'
my_seq, proteins_name = data_prepared(file_name)


In [ ]:
f = open("SAVING FILE NAME", "w")

averaged_vectors=[]
flag=-1


for i in range(len(proteins_name)):
  documents=[]
  print(i)
  # if len(my_seq[i])<=3500:
  documents.append(my_seq[i])
  print(my_seq[i])
  # print((my_seq[i]))


  vectors = [
            # tokenize the document, return it as PyTorch tensors (vectors),
            # and pass it onto the model
            model(**tokenizer(document, return_tensors='pt'))[0].detach().squeeze()
            for document in documents
  ]


  for vector in vectors:
    s = torch.mean(vector, dim=0)
    print(s)
    s=s.numpy()
    #name = f'CAN{i+1}'
    for j in range(len(s)):
      f.write(","+str(s[j]))
    f.write(",")
    f.write(proteins_name[i])
    f.write("\n")

f.close()

# Data

## Uploading OPs and Non-OPs

### ProtBert

In [13]:
# vec_pos: the ProtBert-Based of OPs collected from OP-AND

df_pos = pd.read_csv('/content/drive/MyDrive/Osta/PPP_ProtBERT_embeddings.txt', sep=",", header=None)
df_pos = df_pos.iloc[:, 1:-1]
print(df_pos.shape)
df_pos = df_pos.drop_duplicates()
print(df_pos.shape)

(81, 1024)
(81, 1024)


In [14]:
# vec_neg_Q5T9C2: the ProtBert-Based of NPP collected from Q5T9C2

df_neg1 = pd.read_csv('/content/drive/MyDrive/Osta/NPP_Q5T9C2_ProtBERT_embeddings.txt', sep=",", header=None)
df_neg1 = df_neg1.iloc[:, 1:-1]
df_neg1 = df_neg1.drop_duplicates()

In [15]:
# vec_neg_Q5T9C2: the ProtBert-Based of NPP collected from Q9CWT3

df_neg2 = pd.read_csv('/content/drive/MyDrive/Osta/NPP_Q9CWT3_ProtBERT_embeddings.txt', sep=",", header=None)
df_neg2 = df_neg2.iloc[:, 1:-1]
df_neg2 = df_neg2.drop_duplicates()

In [16]:
# vec_neg_O88942: the ProtBert-Based of NPP collected from O88942

df_neg3 = pd.read_csv('/content/drive/MyDrive/Osta/NPP_O88942_ProtBERT_embeddings.txt', sep=",", header=None)
df_neg3 = df_neg3.iloc[:, 1:-1]
df_neg3 = df_neg3.drop_duplicates()

### ProGen

In [ ]:
# ProGen

# Load the .npz file
data = np.load("/content/drive/MyDrive/Osta/PPP_ProGen2_Medium_embeddings.npz")

print(data.files)  # e.g., ['embedding1', 'embedding2']

df_pos = data['embeddings']
df_pos = pd.DataFrame(df_pos)
df_pos

## Challenge very similar outputs!

In [ ]:
# ProGen

# Load the .npz file
data = np.load("/content/drive/MyDrive/Osta/NPP_Q5T9C2_ProGen2_Medium_embeddings.npz")
df_neg1 = data['embeddings']
df_neg1 = pd.DataFrame(df_neg1_progen)
df_neg1 = df_neg1.drop_duplicates()

In [ ]:
# ProGen

# Load the .npz file
data = np.load("/content/drive/MyDrive/Osta/NPP_Q9CWT3_ProGen2_Medium_embeddings.npz")
df_neg2 = data['embeddings']
df_neg2 = pd.DataFrame(df_neg2_progen)
df_neg2 = df_neg2.drop_duplicates()

In [ ]:
# ProGen

# Load the .npz file
data = np.load("/content/drive/MyDrive/Osta/NPP_O88942_ProGen2_Medium_embeddings.npz")
df_neg3 = data['embeddings']
df_neg3 = pd.DataFrame(df_neg2_progen)
df_neg3 = df_neg3.drop_duplicates()


#### Why the sequences retrieved from ProGen2 are TOO Similar??


1. Mean Pooling Flattens Information

Averaging token (residue) embeddings across the sequence,often washes out rich contextual differences — especially for long or functionally diverse sequences.



2. Causal LM ≠ Good Sequence Embedding

ProGen2 is a causal language model, trained to predict the next amino acid. Its internal representations are not optimized for whole-sequence discrimination like BERT-style models (e.g. ESM-2).

3. Lack of Supervised Signal

Unless ProGen is fine-tuned for a downstream task (e.g., classification, function prediction), its embeddings may not cluster semantically or functionally in a meaningful way.

### ESM-2

In [ ]:
# ESM-2

# Load the .npz file
data = np.load("/content/drive/MyDrive/Osta/PPP_esm2_embeddings.npz")

print(data.files)  # e.g., ['embedding1', 'embedding2']

df_pos = data['embeddings']
df_pos = pd.DataFrame(df_pos)

In [ ]:
# ESM-2

# Load the .npz file
data = np.load("/content/drive/MyDrive/Osta/NPP_Q5T9C2_esm2_embeddings.npz")

df_neg1 = data['embeddings']
df_neg1 = pd.DataFrame(df_neg1_progen)
df_neg1 = df_neg1.drop_duplicates()

In [ ]:
# ESM-2

# Load the .npz file
data = np.load("/content/drive/MyDrive/Osta/NPP_Q9CWT3_esm2_embeddings.npz")

df_neg2 = data['embeddings']
df_neg2 = pd.DataFrame(df_neg2_progen)
df_neg2 = df_neg2.drop_duplicates()

In [ ]:
# ESM-2

# Load the .npz file
data = np.load("/content/drive/MyDrive/Osta/NPP_O88942_esm2_embeddings.npz")

df_neg3 = data['embeddings']
df_neg3 = pd.DataFrame(df_neg3_progen)
df_neg3 = df_neg3.drop_duplicates()

### zScale

In [ ]:
# vec_pos: the ProtBert-Based of OPs collected from OP-AND

df_pos = pd.read_csv('/content/drive/MyDrive/Osta/PPP_zscale.csv', sep=",")
df_pos = df_pos.iloc[:, :-1]
print(df_pos.shape)
df_pos = df_pos.drop_duplicates()
print(df_pos.shape)

(108, 4)
(106, 4)


In [ ]:
# vec_neg_Q5T9C2: the ProtBert-Based of NPP collected from Q5T9C2

df_neg1 = pd.read_csv('/content/drive/MyDrive/Osta/NPP_Q5T9C2_zscale.csv', sep=",")
df_neg1 = df_neg1.iloc[:, :-1]
df_neg1 = df_neg1.drop_duplicates()

In [ ]:
# vec_neg_Q5T9C2: the ProtBert-Based of NPP collected from Q9CWT3

df_neg2 = pd.read_csv('/content/drive/MyDrive/Osta/NPP_Q9CWT3_zscale.csv', sep=",")
df_neg2 = df_neg2.iloc[:, :-1]
df_neg2 = df_neg2.drop_duplicates()

In [ ]:
# vec_neg_O88942: the ProtBert-Based of NPP collected from O88942

df_neg3 = pd.read_csv('/content/drive/MyDrive/Osta/NPP_O88942_zscale.csv', sep=",")
df_neg3 = df_neg3.iloc[:, :-1]
df_neg3 = df_neg3.drop_duplicates()

### tScale

In [ ]:
# vec_pos: the ProtBert-Based of OPs collected from OP-AND

df_pos = pd.read_csv('/content/drive/MyDrive/Osta/PPP_tscale.csv', sep=",")
df_pos = df_pos.iloc[:, :-1]
print(df_pos.shape)
df_pos = df_pos.drop_duplicates()
print(df_pos.shape)

(108, 4)
(106, 4)


In [ ]:
# vec_neg_Q5T9C2: the ProtBert-Based of NPP collected from Q5T9C2

df_neg1 = pd.read_csv('/content/drive/MyDrive/Osta/NPP_Q5T9C2_tscale.csv', sep=",")
df_neg1 = df_neg1.iloc[:, :-1]
df_neg1 = df_neg1.drop_duplicates()

In [ ]:
# vec_neg_Q5T9C2: the ProtBert-Based of NPP collected from Q9CWT3

df_neg2 = pd.read_csv('/content/drive/MyDrive/Osta/NPP_Q9CWT3_tscale.csv', sep=",")
df_neg2 = df_neg2.iloc[:, :-1]
df_neg2 = df_neg2.drop_duplicates()

In [ ]:
# vec_neg_O88942: the ProtBert-Based of NPP collected from O88942

df_neg3 = pd.read_csv('/content/drive/MyDrive/Osta/NPP_O88942_tscale.csv', sep=",")
df_neg3 = df_neg3.iloc[:, :-1]
df_neg3 = df_neg3.drop_duplicates()

### Merging all negs

In [17]:
# merging all neg data of bone loss proteins

df_neg = pd.concat([df_neg1, df_neg2, df_neg3], ignore_index=True)


## Balanced training dataset construction

In [18]:
X1 = df_pos.to_numpy(dtype='float')
X0 = df_neg.to_numpy(dtype='float')


In [19]:
Y1 = [1 for i in range(X1.shape[0])]
Y0 = [0 for i in range(X0.shape[0])]

In [ ]:
x0_train,x0_test,y0_train,y0_test=train_test_split(X0,Y0,test_size=0.2,shuffle=True)
x1_train,x1_test,y1_train,y1_test=train_test_split(X1,Y1,test_size=0.2,shuffle=True)
del X0
del Y0
del X1
del Y1

In [20]:
dataset1p,label1p=makedatasetPos(x1_train)
dataset0p,label0p=makedatasetPos(x0_train)
datasetn,labeln=makedatasetNeg(x0_train,x1_train)
main_train1p=np.asarray(dataset1p)
main_train0p=np.asarray(dataset0p)
main_trainn=np.asarray(datasetn)
main_labeln=np.asarray(labeln)
main_label1p=np.asarray(label1p)
main_label0p=np.asarray(label0p)

del dataset1p
del dataset0p
del datasetn
del labeln
del label1p
del label0p

# SimPep framework

## SimPep-Net

In [21]:
input_shape1=(len(x0_train[0]),)
left_input = Input(shape=(len(x0_train[0]),))
right_input = Input(shape=(len(x0_train[0]),))


hidden_size11=512
hidden_size12=128
hidden_size13=64
hidden_size14=32





model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(hidden_size11,input_shape=input_shape1,  activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(hidden_size12, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(hidden_size13, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(hidden_size14, activation='relu'),
])


encoded_l = model(left_input)
encoded_r = model(right_input)

L1 = tf.keras.layers.Lambda (lambda x: K.abs(x[0]-x[1]))([encoded_l, encoded_r])
L1_D=tf.keras.layers.Dropout(0.2)(L1)
L2 = tf.keras.layers.Dense(16, activation='relu')(L1_D)
L2_D=tf.keras.layers.Dropout(0.2)(L2)
L3 = tf.keras.layers.Dense(16, activation='relu')(L2_D)
L3_D=tf.keras.layers.Dropout(0.2)(L3)
L4 = tf.keras.layers.Dense(16, activation='relu')(L3_D)
L4_D=tf.keras.layers.Dropout(0.2)(L4)
L5 = tf.keras.layers.Dense(8, activation='relu')(L4_D)
L5_D=tf.keras.layers.Dropout(0.2)(L5)
L6 = tf.keras.layers.Dense(1, activation='relu')(L1_D)

prediction = tf.keras.layers.Dense(1, activation='sigmoid')(L1_D)


siamese_net = tf.keras.Model([left_input, right_input], prediction)

optimizer= Adam(learning_rate=0.01)
siamese_net.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy", "mae", "mse"])

optimizer = Adam(learning_rate = 0.0001)
model.compile(loss="kl_divergence",optimizer=optimizer)



plot_model(siamese_net, show_shapes=True, show_layer_names=True)
siamese_net.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1024)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 1024)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 32)        │    600,800 │ input_layer[0][0… │
│ (Sequential)        │                   │            │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 32)        │          0 │ sequential[0][0], │
│                     │                   │            │ sequential[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 32)        │          0 │ lambda[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 1)         │         33 │ dropout_3[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 600,833 (2.29 MB)

 Trainable params: 600,833 (2.29 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# # COLLAPPI inspired

# input_shape1=(len(x0_train[0]),)
# left_input = Input(shape=(len(x0_train[0]),))
# right_input = Input(shape=(len(x0_train[0]),))


# hidden_size11=512
# hidden_size12=128
# hidden_size13=64
# hidden_size14=32




# model = tf.keras.models.Sequential([
#     tf.keras.layers.Dense(hidden_size11,input_shape=input_shape1,  activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(hidden_size12, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(hidden_size13, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(hidden_size14, activation='relu'),
# ])


# encoded_l = model(left_input)
# encoded_r = model(right_input)

# L1 = tf.keras.layers.Lambda (lambda x: K.abs(x[0]-x[1]))([encoded_l, encoded_r])
# L1_D=tf.keras.layers.Dropout(0.2)(L1)
# # Now we add CollaPPI-inspired MLP head:
# MLP1 = tf.keras.layers.Dense(512, activation='relu')(L1_D)
# MLP1_D = tf.keras.layers.Dropout(0.2)(MLP1)

# MLP2 = tf.keras.layers.Dense(256, activation='relu')(MLP1_D)
# MLP2_D = tf.keras.layers.Dropout(0.2)(MLP2)

# MLP3 = tf.keras.layers.Dense(1)(MLP2_D)  # linear output first
# prediction = tf.keras.layers.Activation('sigmoid')(MLP3)  # sigmoid final activation

# # Build model:
# siamese_net = tf.keras.Model([left_input, right_input], prediction)

# optimizer= Adam(learning_rate=0.0001)
# siamese_net.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=["accuracy", "mae", "mse"])

# optimizer = Adam(learning_rate = 0.0001)
# model.compile(loss="kl_divergence",optimizer=optimizer)



# plot_model(siamese_net, show_shapes=True, show_layer_names=True)
# siamese_net.summary()

## Training optimization

In [22]:
from tensorflow.keras.callbacks import  History
Epoch=5
history=History()
train,label = MakeDatasetforTrain(main_train1p,main_train0p,main_trainn,main_label1p,main_label0p,main_labeln)
main_trainL1=np.array(train[:,0:int(len(train[0])/2)])
main_trainR1=np.array(train[:,int(len(train[0])/2):len(train[0])])
main_label1=np.array(label)
count1 = np.count_nonzero(main_label1 == 1)
count0 = np.count_nonzero(main_label1 == 0)
print(count1 , count0)

P = siamese_net.fit([main_trainL1, main_trainR1],main_label1, epochs= Epoch,batch_size=128, callbacks=history)
for j in range(1000):
  C=1
  Before=int(P.history['accuracy'][-1]*100)
  for i in range(2,Epoch+1):
    if  int(P.history['accuracy'][-i]*100)==Before:
      C=C+1
    else:
      C=1
    Before=int(P.history['accuracy'][-i]*100)
    print(Before)
  if C==5:
    break

  train,label=MakeDatasetforTrain(main_train1p,main_train0p,main_trainn,main_label1p,main_label0p,main_labeln)
  main_trainL1=np.array(train[:,0:int(len(train[0])/2)])
  main_trainR1=np.array(train[:,int(len(train[0])/2):len(train[0])])
  main_label1=np.array(label)
  P = siamese_net.fit([main_trainL1, main_trainR1], main_label1,batch_size=128, epochs=Epoch, callbacks=history)
  print(j+1)

P1 3240 P0 3240
6480 6480
Epoch 1/5
102/102 ━━━━━━━━━━━━━━━━━━━━ 13s 48ms/step - accuracy: 0.5106 - loss: 0.7105 - mae: 0.4997 - mse: 0.2573
Epoch 2/5
102/102 ━━━━━━━━━━━━━━━━━━━━ 13s 76ms/step - accuracy: 0.5019 - loss: 0.6941 - mae: 0.4998 - mse: 0.2505
Epoch 3/5
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.5262 - loss: 0.6891 - mae: 0.4961 - mse: 0.2480
Epoch 4/5
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step - accuracy: 0.6939 - loss: 0.6054 - mae: 0.4246 - mse: 0.2084
Epoch 5/5
102/102 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.8435 - loss: 0.4076 - mae: 0.2636 - mse: 0.1233
74
56
50
50
P1 3240 P0 3240
Epoch 1/5
102/102 ━━━━━━━━━━━━━━━━━━━━ 3s 25ms/step - accuracy: 0.9103 - loss: 0.2717 - mae: 0.1671 - mse: 0.0744
Epoch 2/5
102/102 ━━━━━━━━━━━━━━━━━━━━ 5s 27ms/step - accuracy: 0.9013 - loss: 0.2884 - mae: 0.1637 - mse: 0.0790
Epoch 3/5
102/102 ━━━━━━━━━━━━━━━━━━━━ 6s 32ms/step - accuracy: 0.9321 - loss: 0.2184 - mae: 0.1229 - mse: 0.0555
Epoch 4/5
102/102 ━━━━━━━━━━━━━━

In [25]:
siamese_net.save('/content/drive/MyDrive/Osta/full_model.keras')

In [28]:
siamese_net.save_weights('/content/drive/MyDrive/Osta/weights_only.weights.h5')

## Base ML Models: RF and SVM

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.svm import SVC
# from sklearn.ensemble import RandomForestClassifier


# svm_model = SVC(kernel='rbf')
# svm_model.fit(train,label)

# rf_model = RandomForestClassifier(n_estimators=100)
# rf_model.fit(train,label)

RandomForestClassifier()

In [ ]:
# removing unnecceasary variables to reduce the RAM usage
del df_neg, df_pos, main_train0p, main_label1, main_label0p, main_label1p, main_trainL1, main_trainR1, main_train1p

## SimPep-Net validation

In [ ]:
main_test2,main_label_test2=InternalTest(x1_test,x0_test)
y_pred=siamese_net.predict([np.array(main_test2[:,0:int(len(main_test2[0])/2)]),np.array(main_test2[:,int(len(main_test2[0])/2):len(main_test2[0])])])
Accuracy(main_label_test2, y_pred)
# del main_test2,main_label_test2, y_pred


220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
AUC:  0.594430281819927 AUPR:  0.7417293590609846
ACC: 60.17661301808859 Spc: 0.5009372071227741 Sen:  0.6457949662369552 TN: 1069 FP:  1065 FN 1731 TP 3156


In [ ]:
# y_pred_rf = rf_model.predict(main_test2)
# Accuracy(main_label_test2, y_pred_rf)

# y_pred_svm = svm_model.predict(main_test2)
# Accuracy(main_label_test2, y_pred_svm)

## OPD Prediction

In [ ]:
main_test1,main_label_test1,Indexsample=ExternalTest(x0_test,x1_test,x0_train,x1_train)
y_pred=siamese_net.predict([np.array(main_test1[:,0:int(len(main_test1[0])/2)]),np.array(main_test1[:,int(len(main_test1[0])/2):len(main_test1[0])])])
Pred,Label=DefineTest2(x0_test,x1_test,Indexsample,y_pred)
# del main_test1,main_label_test1,Indexsample
Accuracy(Label, Pred)
# del Label, Pred

1745/1745 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step
AUC:  0.6740862230552953 AUPR:  0.3676730653588882
ACC: 73.10924369747899 Spc: 0.7835051546391752 Sen:  0.5 TN: 76 FP:  21 FN 11 TP 11


In [ ]:
# y_pred_rf = rf_model.predict(main_test1)
# Pred,Label=DefineTest2(x0_test,x1_test,Indexsample,y_pred_rf)
# Accuracy(Label, Pred)


# y_pred_svm = svm_model.predict(main_test1)
# Pred,Label=DefineTest2(x0_test,x1_test,Indexsample,y_pred_svm)
# Accuracy(Label, Pred)

AUC:  0.5204728950403691 AUPR:  0.3138528138528139
ACC: 86.5546218487395 Spc: 1.0 Sen:  0.058823529411764705 TN: 102 FP:  0 FN 16 TP 1
AUC:  0.6384083044982699 AUPR:  0.3294280426292972
ACC: 85.71428571428571 Spc: 0.9901960784313726 Sen:  0.058823529411764705 TN: 101 FP:  1 FN 16 TP 1


In [ ]:
## Checking for case study



# df_test = pd.read_csv('vec_case_study.txt', sep="\t", header=None)
# X=np.asarray(df_test)



# Load the .npz file
data = np.load("/content/drive/MyDrive/Osta/camel_esm2_embeddings.npz")

print(data.files)

df_test = data['embeddings']
df_test = pd.DataFrame(df_test)
X=np.asarray(df_test)


['embeddings', 'labels', 'sequences']


In [ ]:
main_test,IndexsampleTest=RealTest(X,x0_train,x1_train)
main_tesxLX=np.array(main_test[:,0:int(len(main_test[0])/2)])
main_testRX=np.array(main_test[:,int(len(main_test[0])/2):len(main_test[0])])
y_pred=siamese_net.predict([main_tesxLX,main_testRX])
y_pred=RealPredict(X,y_pred,IndexsampleTest)
SelectName(y_pred,0.8)

246/246 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
human-alpha s1-peptide 8 [0.81445456]
{'human-alpha s1-peptide 8': array([0.81445456], dtype=float32)}


In [ ]:
del y_pred, X, main_tesxLX,main_testRX

In [ ]:
del IndexsampleTest, history, label, main_trainn



# External test sets


In [ ]:
# Experminetal_negative_vec: the ProtBert-based vector of experimental negative data

df_test2 = pd.read_csv('Experminetal_negative_vec.txt', header=None)

X_new_neg = np.asarray(df_test2)


In [ ]:
main_test,IndexsampleTest=RealTest(X_new_neg,x0_train,x1_train)
main_tesxLX=np.array(main_test[:,0:int(len(main_test[0])/2)])
main_testRX=np.array(main_test[:,int(len(main_test[0])/2):len(main_test[0])])
y_pred=siamese_net.predict([main_tesxLX,main_testRX])
y_pred=RealPredict(X_new_neg,y_pred,IndexsampleTest)
y_pred

[array([0.22340016], dtype=float32),
 array([0.13297284], dtype=float32),
 array([0.10937601], dtype=float32),
 array([0.88242143], dtype=float32)]

In [ ]:
# new_OP_vec: the ProtBert-based vector of Ops from 2022 to 2024
# For this assessment, the model should be trained first without this data

df_test = pd.read_csv('new_OP_vec.txt', sep="\t", header=None)
X_new_pos=np.asarray(df_test)

In [ ]:
main_test,IndexsampleTest=RealTest(X_new_pos,x0_train,x1_train)
main_tesxLX=np.array(main_test[:,0:int(len(main_test[0])/2)])
main_testRX=np.array(main_test[:,int(len(main_test[0])/2):len(main_test[0])])

In [ ]:
y_pred=siamese_net.predict([main_tesxLX,main_testRX])
y_pred=RealPredict(X_new_pos,y_pred,IndexsampleTest)
y_pred

[array([0.93163073], dtype=float32),
 array([0.2837767], dtype=float32),
 array([0.17874585], dtype=float32),
 array([0.16910979], dtype=float32),
 array([0.17543823], dtype=float32),
 array([0.15224825], dtype=float32),
 array([0.1593636], dtype=float32),
 array([0.85469], dtype=float32),
 array([0.89563197], dtype=float32),
 array([0.87257445], dtype=float32),
 array([0.8969664], dtype=float32),
 array([0.9104396], dtype=float32),
 array([0.9180465], dtype=float32),
 array([0.12953553], dtype=float32),
 array([0.86453176], dtype=float32),
 array([0.18906485], dtype=float32),
 array([0.18737742], dtype=float32),
 array([0.17875192], dtype=float32),
 array([0.5052275], dtype=float32),
 array([0.20020354], dtype=float32),
 array([0.2154391], dtype=float32),
 array([0.16753079], dtype=float32),
 array([0.50195324], dtype=float32),
 array([0.27825797], dtype=float32),
 array([0.18382709], dtype=float32),
 array([0.15432584], dtype=float32),
 array([0.16753079], dtype=float32)]